<a href="https://colab.research.google.com/github/trucabrac/blob_Jan2022/blob/main/Blob_batch_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import os
import glob
from skimage.filters import gaussian
from skimage import img_as_ubyte
import random
from google.colab.patches import cv2_imshow
import csv

#1. Read images and store them in an array

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/blob-tl/

Mounted at /content/drive
/content/drive/MyDrive/blob-tl


In [ ]:
%cd tl-selec/tl10/
#%cd ../tl9/
%ls

/content/drive/MyDrive/blob-tl/tl-selec/tl10
tl10-018.jpeg  tl10-126.jpeg  tl10-207.jpeg  tl10-315.jpeg
tl10-045.jpeg  tl10-153.jpeg  tl10-234.jpeg  tl10-342.jpeg
tl10-072.jpeg  tl10-180.jpeg  tl10-261.jpeg  tl10-369.jpeg
tl10-099.jpeg  tl10-200.jpeg  tl10-288.jpeg  tl10-396.jpeg


In [ ]:
#################################################
#Capture all mages into an array and then iterate through each image
#Normally used for machine learning workflows.

images_list = []
images_names = []
SIZE = 512

path = "*.*"
#pathOut = "test-clas/"
pathOut = "../tl-contour/" #folder to create beforehand
#path = "tl4-proc/*.*"
#pathOut = "tl4-clas/"
#label = 'tl4-'

#First create a stack array of all images
for file in glob.glob(path):
    print(file)     #just stop here to see all file names printed
    img0= cv2.imread(file, 0)  #now, we can read each file since we have the full path
    #img = cv2.cvtColor(imgIn, cv2.IMREAD_GRAYSCALE)
    #img = cv2.resize(img, (SIZE, SIZE))
    images_list.append(img0)
    images_names.append(file)
        
images_list = np.array(images_list)    

tl10-018.jpeg
tl10-045.jpeg
tl10-072.jpeg
tl10-099.jpeg
tl10-126.jpeg
tl10-153.jpeg
tl10-180.jpeg
tl10-200.jpeg
tl10-207.jpeg
tl10-234.jpeg
tl10-261.jpeg
tl10-288.jpeg
tl10-315.jpeg
tl10-342.jpeg
tl10-369.jpeg
tl10-396.jpeg


#2. Import Keras ImageNet models

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions

#3. Define functions

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX

def getContours(im,word):
  (hc, wc) = im.shape[:2]
  x1 = wc/2
  x2=wc/2
  y1=hc/2
  y2=hc/2
  
  contours,hierarchy = cv2.findContours(im, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
  for cnt in contours:
    area= cv2.contourArea(cnt)
    if area>0:
      imgMid=cv2.drawContours(imgContour,cnt,-1,(0,255,0),3)
      #peri = cv2.arcLength(cnt,True)
      #approx = cv2.approxPolyDP(cnt,) 
    
      #draw bounding rectangles
      x,y,w,h = cv2.boundingRect(cnt)
      if x < x1: x1 = x-20
      if x+w > x2: x2 = x+w+20
      if y < y1: y1 = y-20
      if y+h > y2: y2 = y+h+20
  imgFinal = cv2.rectangle(imgMid,(x1,y1),(x2,y2),(255,0,0),6)
  cv2.putText(imgFinal, word, (x1, y1-20), font, 2, (255,0,0), 9)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX

#alternative without bounding box and word
def getContours2(im):
  (hc, wc) = im.shape[:2]
  x1 = wc/2
  x2=wc/2
  y1=hc/2
  y2=hc/2
  
  contours,hierarchy = cv2.findContours(im, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
  for cnt in contours:
    area= cv2.contourArea(cnt)
    if area>0:
      imgMid=cv2.drawContours(imgContour,cnt,-1,(0,255,0),3)
      #peri = cv2.arcLength(cnt,True)
      #approx = cv2.approxPolyDP(cnt,) 
    
  imgFinal = imgMid


In [ ]:
#run just once when creating the csv
f = open('/content/drive/MyDrive/blob-tl/blobtl-clas-prob-top3.csv', 'w')
# create the csv writer
writer = csv.writer(f)
# write the header
#header = ['imgpath', 'class']
writer.writerow(['imgpath', 'class'])
f.close()

In [ ]:
# open the csv file in the write mode to store classifications
f = open('/content/drive/MyDrive/blob-tl/blobtl-clas-prob-top3.csv', 'a')
# create the csv writer
writer = csv.writer(f)

#4. Process each image -> classif + text

In [ ]:
#Process each image in the array
img_number = 0
for image in range(images_list.shape[0]):
    inImg = images_list[image,:,:]  #Grey images. For color add another dim.
    #smoothed_image = img_as_ubyte(gaussian(inImg, sigma=5, mode='constant', cval=0.0))

    #preprocess img 
    dim = (224, 224)
    sizImg = cv2.resize(inImg, dim, interpolation=cv2.INTER_LINEAR)
    #cv2_imshow(sizImg)
    x = cv2.cvtColor(sizImg, cv2.COLOR_GRAY2RGB)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)


    #Classify with Keras models
    #call randomly one of the models
    nModel = random.randint(0,2)
    if nModel==0:
      model_vgg16 = VGG16(weights='imagenet')
      preds = model_vgg16.predict(x)
    if nModel==1:
      model_rn50 = ResNet50(weights='imagenet')
      preds = model_rn50.predict(x)
    if nModel==2:
      model_mobilenet = MobileNet(weights='imagenet')
      preds = model_mobilenet.predict(x)
    # decode the results into a list of tuples (class, description, probability)
    # and choose 1 result randomly from the top20
    pred_top = decode_predictions(preds, top=20)[0]
    #print('Predicted:', pred_top)
    topn = random.randint(0,19)
    #print(topn)
    pred_dis = pred_top[topn][1]
    #print('Predicted:', pred_dis)

    #find contours and draw bounding box in image
    imgCanny = cv2.Canny(inImg,500,500)
    #cv2_imshow(imgCanny)
    imgContour = cv2.cvtColor(inImg, cv2.COLOR_GRAY2RGB)
    getContours(imgCanny,pred_dis)
    #cv2_imshow(imgContour)

    imgPath = pathOut+label+str(img_number)
    #save image with contour
    cv2.imwrite(pathOut+label+str(img_number)+".jpg", imgContour)
    #store img ref and class in csv
    #writer.writerow([imgPath, pred_dis])

    #increment
    img_number +=1   

NameError: ignored

In [ ]:
# Process images contours only
#Process each image in the array
img_number = 0
label = 'test'
for image in range(images_list.shape[0]):
    inImg = images_list[image,:,:]  #Grey images. For color add another dim.
    #smoothed_image = img_as_ubyte(gaussian(inImg, sigma=5, mode='constant', cval=0.0))

    #find contours and draw them
    imgCanny = cv2.Canny(inImg,300,300)
    #cv2_imshow(imgCanny)
    imgContour = cv2.cvtColor(inImg, cv2.COLOR_GRAY2RGB)
    getContours2(imgCanny)
    #cv2_imshow(imgContour)

    #imgPath = pathOut+images_names[image]+str(img_number)
    #save image with contour
    #cv2.imwrite(pathOut+label+str(img_number)+".jpg", imgContour)
    cv2.imwrite(pathOut+images_names[image], imgContour)
    #store img ref and class in csv
    #writer.writerow([imgPath, pred_dis])

    #increment
    img_number +=1   

In [ ]:
#Process Keras classifications only
img_number = 0
for image in range(images_list.shape[0]):
    inImg = images_list[image,:,:]  #Grey images. For color add another dim.
    #smoothed_image = img_as_ubyte(gaussian(inImg, sigma=5, mode='constant', cval=0.0))

    #preprocess img 
    dim = (224, 224)
    sizImg = cv2.resize(inImg, dim, interpolation=cv2.INTER_LINEAR)
    #cv2_imshow(sizImg)
    x = cv2.cvtColor(sizImg, cv2.COLOR_GRAY2RGB)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)


    #Classify with Keras models
    #call randomly one of the models
    nModel = random.randint(0,2)
    if nModel==0:
      model_vgg16 = VGG16(weights='imagenet')
      preds = model_vgg16.predict(x)
    if nModel==1:
      model_rn50 = ResNet50(weights='imagenet')
      preds = model_rn50.predict(x)
    if nModel==2:
      model_mobilenet = MobileNet(weights='imagenet')
      preds = model_mobilenet.predict(x)
    # decode the results into a list of tuples (class, description, probability)
    # and choose 1 result randomly from the top20
    pred_top = decode_predictions(preds, top=3)[0]
    #print('Predicted:', pred_top)
    topn = random.randint(0,2)
    #print(topn)
    pred_dis = pred_top[topn][1]
    pred_p = pred_top[topn][2] * 100
    #print('Predicted:', pred_dis)

    imgPath = pathOut+label+str(img_number)
    proba = str(pred_p)+'%'
    #save image with contour
    #cv2.imwrite(pathOut+images_names[image]+".jpg", imgContour)
    #store img ref and class in csv
    writer.writerow([images_names[image], pred_dis, proba])

    #increment
    img_number +=1   

102981632/102967424 [==============================] - 1s 0us/step


In [ ]:
# close the file
f.close()